In [2]:
# =========================
# CS-340 Project Two Dashboard (no dash_leaflet) — WORKING
# =========================

from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px

import os, base64
import numpy as np
import pandas as pd

# ---- CRUD module (wrapper you created earlier) ----
from animal_shelter import AnimalShelter

# ========= DB connection (YOUR creds) =========
db = AnimalShelter("aacuser", "CS340!Demo2025", "localhost", 27017, "aac", "animals")

# ========= Helper: tolerate different read() signatures =========
def mongo_read(db, query=None, projection=None):
    query = query or {}
    try:
        return list(db.read(query, projection)) if projection is not None else list(db.read(query))
    except TypeError:
        return list(db.read(query))

# ========= Small utilities =========
def age_to_months(age_str):
    """Convert '2 years'/'6 months'/'3 weeks'/'10 days' -> months (float). Unknown -> inf."""
    try:
        if not isinstance(age_str, str):
            return float("inf")
        parts = age_str.lower().split()
        if len(parts) < 2:
            return float("inf")
        n = float(parts[0])
        unit = parts[1]
        if "year" in unit: return n * 12.0
        if "month" in unit: return n
        if "week" in unit: return n / 4.0
        if "day"  in unit: return n / 30.0
        return float("inf")
    except Exception:
        return float("inf")

# Rescue types -> preferred breeds
RESCUE_BREED_MAP = {
    "Water Rescue": ["Labrador Retriever", "Newfoundland", "Chesapeake Bay Retriever"],
    "Mountain or Wilderness Rescue": ["German Shepherd", "Border Collie", "Bloodhound"],
    "Disaster or Individual Tracking": ["Doberman Pinscher", "German Shepherd", "Golden Retriever"],
}

def _series_or_nan(df: pd.DataFrame, col: str) -> pd.Series:
    return df[col] if col in df.columns else pd.Series([np.nan] * len(df), index=df.index)

def fetch_df(rescue_type=None, only_age_le_two=True):
    """Fetch dogs, apply optional rescue-type (breed) and age filters, unify lat/lon."""
    query = {"animal_type": "Dog"}
    if rescue_type and rescue_type in RESCUE_BREED_MAP:
        query["$or"] = [{"breed": {"$regex": b, "$options": "i"}} for b in RESCUE_BREED_MAP[rescue_type]]

    projection = {
        "_id": 0,
        "name": 1,
        "breed": 1,
        "animal_type": 1,
        "sex_upon_outcome": 1,
        "age_upon_outcome": 1,
        "outcome_type": 1,
        "location_lat": 1,
        "location_long": 1,
        "outcome_latitude": 1,
        "outcome_longitude": 1
    }

    rows = mongo_read(db, query, projection)
    df_local = pd.DataFrame(rows)
    if df_local.empty:
        return df_local

    # unify coordinates (prefer location_*; fallback to outcome_*)
    loc_lat = _series_or_nan(df_local, "location_lat")
    out_lat = _series_or_nan(df_local, "outcome_latitude")
    loc_lon = _series_or_nan(df_local, "location_long")
    out_lon = _series_or_nan(df_local, "outcome_longitude")

    df_local["lat"] = loc_lat.where(loc_lat.notna(), out_lat)
    df_local["lon"] = loc_lon.where(loc_lon.notna(), out_lon)

    # optional age filter (≤ 24 months)
    if only_age_le_two and "age_upon_outcome" in df_local.columns:
        df_local["age_months"] = df_local["age_upon_outcome"].apply(age_to_months)
        df_local = df_local[df_local["age_months"] <= 24.0].copy()

    return df_local

# Initial dataset
df_init = fetch_df(rescue_type=None, only_age_le_two=True)

# ========= View / Layout =========
app = JupyterDash(__name__)

# Optional: logo (put 'Grazioso Salvare Logo.png' in the same folder) OR leave text fallback
logo_path = "Grazioso Salvare Logo.png"
if os.path.exists(logo_path):
    encoded_image = base64.b64encode(open(logo_path, 'rb').read()).decode("utf-8")
    logo_img = html.Img(src=f"data:image/png;base64,{encoded_image}", style={"height": "60px"})
else:
    logo_img = html.Div("Grazioso Salvare", style={"fontWeight": "bold", "fontSize": "20px"})

app.layout = html.Div([
    # Header with logo + identifier (PUT YOUR NAME BELOW)
    html.Div([
        logo_img,
        html.Div("Your Name — CS-340 Project Two", style={"fontSize": "12px", "marginTop": "4px"})
    ], style={"display": "inline-block", "verticalAlign": "top", "marginRight": "16px"}),

    html.Center(html.B(html.H1("CS-340 Dashboard"))),
    html.Hr(),

    # Filters
    html.Div([
        html.Div([
            html.Label("Rescue Type"),
            dcc.Dropdown(
                id='filter-type',
                options=[{"label": k, "value": k} for k in RESCUE_BREED_MAP.keys()],
                placeholder="Select a rescue type",
                clearable=True
            ),
        ], style={'minWidth': '280px', 'marginRight': '16px'}),

        html.Div([
            html.Label("Age Filter"),
            dcc.Checklist(
                id='age-filter',
                options=[{"label": "Only dogs ≤ 2 years old", "value": "AGE2"}],
                value=["AGE2"]
            ),
        ], style={'minWidth': '260px', 'marginRight': '16px'}),

        html.Button("Reset", id="reset-btn", n_clicks=0, style={'height': '38px', 'marginTop': '22px'})
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '12px'}),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True}
                 for c in (df_init.columns if not df_init.empty else
                           ["name","breed","age_upon_outcome","sex_upon_outcome","outcome_type","lat","lon"])],
        data=(df_init.to_dict('records') if not df_init.empty else []),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        style_table={"overflowX": "auto"},
        style_cell={"fontFamily": "Arial", "fontSize": 12, "whiteSpace": "normal", "height": "auto"}
    ),

    html.Br(), html.Hr(),

    # Charts row (left: bar chart, right: map)
    html.Div(className='row',
             style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '16px'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6', style={'flex': '1 1 50%'}),
                 html.Div(id='map-id', className='col s12 m6', style={'flex': '1 1 50%'})
             ])
])

# ========= Controller / Callbacks =========

# Update DataTable (data + columns) from filters
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    Input('filter-type', 'value'),
    Input('age-filter', 'value'),
    Input('reset-btn', 'n_clicks'),
    prevent_initial_call=False
)
def update_dashboard(filter_type, age_vals, n_clicks):
    # Handle Reset
    triggered = [t['prop_id'] for t in dash.callback_context.triggered] if hasattr(dash, "callback_context") else []
    if triggered and 'reset-btn.n_clicks' in triggered[0]:
        filter_type = None
        age_vals = ["AGE2"]

    only_age2 = "AGE2" in (age_vals or [])
    dff = fetch_df(rescue_type=filter_type, only_age_le_two=only_age2)

    cols = [{"name": c, "id": c, "deletable": False, "selectable": True}
            for c in (dff.columns if not dff.empty else
                      ["name","breed","age_upon_outcome","sex_upon_outcome","outcome_type","lat","lon"])]
    data = dff.to_dict('records') if not dff.empty else []
    return data, cols

# Chart: top breeds from current DataTable view
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData is not None else df_init.copy()
    if dff.empty or "breed" not in dff.columns:
        fig = px.bar(pd.DataFrame({"breed": [], "count": []}), x="breed", y="count", title="Breeds (Count)")
    else:
        top = dff["breed"].fillna("Unknown").value_counts().nlargest(15).reset_index()
        top.columns = ["breed", "count"]
        fig = px.bar(top, x="breed", y="count", title="Breeds (Top 15)")
        fig.update_layout(xaxis_tickangle=-45, margin=dict(l=16, r=16, t=40, b=120))
    return [dcc.Graph(figure=fig)]

# Map using Plotly scatter_mapbox (no extra packages; uses OpenStreetMap)
@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame(viewData) if viewData is not None else df_init.copy()

    # Choose columns for lat/lon (whichever exist)
    lat_col = next((c for c in ["lat", "location_lat", "outcome_latitude"] if c in dff.columns), None)
    lon_col = next((c for c in ["lon", "location_long", "outcome_longitude"] if c in dff.columns), None)

    # If no coords, render a friendly placeholder
    if dff.empty or not lat_col or not lon_col:
        placeholder = px.scatter_mapbox(pd.DataFrame({"lat":[30.75], "lon":[-97.48]}),
                                        lat="lat", lon="lon", zoom=9,
                                        title="Map (no coordinates available yet)")
        placeholder.update_layout(mapbox_style="open-street-map", margin=dict(l=0, r=0, t=40, b=0))
        return [dcc.Graph(figure=placeholder)]

    # If a row is selected, center on it; otherwise plot all valid points
    dff_geo = dff.copy()
    # coerce coords to numeric and drop NaNs
    dff_geo[lat_col] = pd.to_numeric(dff_geo[lat_col], errors="coerce")
    dff_geo[lon_col] = pd.to_numeric(dff_geo[lon_col], errors="coerce")
    dff_geo = dff_geo.dropna(subset=[lat_col, lon_col])

    if dff_geo.empty:
        placeholder = px.scatter_mapbox(pd.DataFrame({"lat":[30.75], "lon":[-97.48]}),
                                        lat="lat", lon="lon", zoom=9,
                                        title="Map (no valid coordinates in current view)")
        placeholder.update_layout(mapbox_style="open-street-map", margin=dict(l=0, r=0, t=40, b=0))
        return [dcc.Graph(figure=placeholder)]

    # Single selection defaults to first row
    row = (selected_rows[0] if selected_rows else 0)
    row = min(max(row, 0), len(dff_geo) - 1)

    # Plot all points; highlight selected with larger size
    dff_geo["_is_selected"] = False
    dff_geo.iloc[row, dff_geo.columns.get_loc("_is_selected")] = True

    fig = px.scatter_mapbox(
        dff_geo,
        lat=lat_col, lon=lon_col,
        color="_is_selected",
        size="_is_selected",
        size_max=15,
        hover_name=dff_geo.get("name") if "name" in dff_geo.columns else None,
        hover_data={"breed": True} if "breed" in dff_geo.columns else None,
        zoom=10,
        title="Animal Locations"
    )
    fig.update_traces(marker={'sizemin': 6})
    fig.update_layout(mapbox_style="open-street-map", showlegend=False,
                      margin=dict(l=0, r=0, t=40, b=0))
    return [dcc.Graph(figure=fig)]

# Run app in JupyterLab mode (change port if needed)
app.run_server(mode='jupyterlab')


In [11]:
import os, re, json

SEARCH_FOR = re.compile(r"animal_shelter\.py$", re.I)   # case-insensitive file match
FOUND = []

for root, dirs, files in os.walk(os.getcwd()):
    for f in files:
        if SEARCH_FOR.search(f):
            FOUND.append(os.path.join(root, f))

print("CWD:", os.getcwd())
print("Found candidates:")
for p in FOUND:
    print("  -", p)

if not FOUND:
    print("\nNo file named 'animal_shelter.py' found. Check if your Project One used a different file name (e.g., CRUD.py, AnimalShelter.py).")


CWD: /home/codio/workspace/code_files
Found candidates:

No file named 'animal_shelter.py' found. Check if your Project One used a different file name (e.g., CRUD.py, AnimalShelter.py).


In [16]:
# Create a minimal CRUD module named animal_shelter.py next to this notebook

import os, sys, textwrap, subprocess

# Ensure pymongo is available
try:
    import pymongo  # noqa: F401
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pymongo"])

code = textwrap.dedent("""
    # animal_shelter.py
    # Minimal CRUD class for CS-340 dashboards.
    from typing import Any, Dict, Iterable, Optional
    from pymongo import MongoClient
    from pymongo.errors import ConnectionFailure

    class AnimalShelter:
        def __init__(self, user: str, password: str,
                     host: str = "localhost", port: int = 27017,
                     db: str = "aac", collection: str = "animals"):
            # Build a simple mongodb URI; password is not URL-encoded here because you're using localhost
            uri = f"mongodb://{user}:{password}@{host}:{port}"
            self.client = MongoClient(uri, serverSelectionTimeoutMS=5000)
            try:
                self.client.admin.command("ping")
            except ConnectionFailure as e:
                raise RuntimeError(f"Cannot connect to MongoDB at {host}:{port} as {user}: {e}")
            self.database = self.client[db]
            self.col = self.database[collection]

        # Create
        def create(self, data: Dict[str, Any]) -> bool:
            if data and isinstance(data, dict):
                res = self.col.insert_one(data)
                return res.acknowledged
            return False

        # Read (returns a cursor/iterable; wrap with list() or DataFrame)
        def read(self, query: Optional[Dict[str, Any]] = None,
                 projection: Optional[Dict[str, int]] = None) -> Iterable[Dict[str, Any]]:
            query = query or {}
            return self.col.find(query, projection)

        # Update
        def update(self, query: Dict[str, Any], new_values: Dict[str, Any], many: bool = False) -> int:
            if many:
                res = self.col.update_many(query, {"$set": new_values})
            else:
                res = self.col.update_one(query, {"$set": new_values})
            return res.modified_count

        # Delete
        def delete(self, query: Dict[str, Any], many: bool = False) -> int:
            if many:
                res = self.col.delete_many(query)
            else:
                res = self.col.delete_one(query)
            return res.deleted_count
""")

with open("animal_shelter.py", "w") as f:
    f.write(code)

print("✅ Created animal_shelter.py in:", os.getcwd())
print("📄 Files here now:", os.listdir())


✅ Created animal_shelter.py in: /home/codio/workspace/code_files
📄 Files here now: ['Grazioso Salvare Logo.png', 'ProjectOneTestScript.ipynb', 'ModuleFiveAssignment.ipynb', 'CRUD_Python_Module.py', 'ProjectTwoDashboard.ipynb', 'ModuleFourTestScript.ipynb', 'ModuleSixMilestone.ipynb', 'Tutorial_SampleCode.ipynb', '.ipynb_checkpoints', '__pycache__', 'Untitled.ipynb', 'Untitled1.ipynb', 'CRUD_PYTHON_5PROJECTONE.py', 'CRUD_PYTHON_MODULE5.py', 'Untitled Folder', 'crudpython_module6.py', 'animal_shelter.py']


In [28]:
pip install dash-leaflet


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
